In [1]:
%run common.ipynb

tokenizer.decode(tokenizer.get_data(third_number=True))

'S184.04=97.13--37.65+49.26E'

In [2]:
model_dpo = torch.load('dpo.model')
model_dpo.to(device)
model_dpo.eval()

/root/anaconda3/envs/pt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelGEN(
  (feature): LlamaModel(
    (embed_tokens): Embedding(22, 64)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=64, out_features=64, bias=False)
          (k_proj): Linear(in_features=64, out_features=64, bias=False)
          (v_proj): Linear(in_features=64, out_features=64, bias=False)
          (o_proj): Linear(in_features=64, out_features=64, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=64, out_features=64, bias=False)
          (up_proj): Linear(in_features=64, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=64, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (fc_out): Linear(in_features=64, out_features=22

In [3]:
#随机一批数据
input_ids = [tokenizer.get_data(third_number=True) for i in range(64)]

#切分成question和answer
split = [i.index(tokenizer.encoder['=']) + 1 for i in input_ids]
question = [input_ids[i][:split[i]] for i in range(len(input_ids))]
answer = [input_ids[i][split[i]:] for i in range(len(input_ids))]

#根据question生成predict
input_ids = [torch.LongTensor(i).unsqueeze(0).to(device) for i in question]
predict = [generate(model_dpo, i) for i in input_ids]

#裁剪,只要生成的部分
predict = [p[0].tolist()[len(q):] for p, q in zip(predict, question)]

#解码成文本
question = [tokenizer.decode(i) for i in question]
answer = [tokenizer.decode(i) for i in answer]
predict = [tokenizer.decode(i) for i in predict]

for q, a, p in zip(question, answer, predict):
    try:
        diff = abs(float(q[1:-1]) - eval(p[:p.index('E')]))
    except:
        diff = abs(float(q[1:-1]))

    print(q, p, diff)

S59.08= -43.54/-96.30+58.20E 0.42787123572169605
S86.25= -76.00+68.12+99.21E 5.079999999999998
S399.59= 94.09*4.18+78.57E 72.27620000000002
S61.42= 93.88+9.01+-53.59E 12.120000000000005
S63.39= -79.70/-96.54+62.59E 0.025564532836135356
S-4389.33= 49.30*-86.85+-8.44E 99.1850000000004
S-6603.37= -78.70*81.01+-74.80E 153.08299999999872
S-130.09= -97.88+-37.18+11.58E 6.609999999999999
S9.26= 1.06+99.14+-70.34E 20.6
S61.99= 91.78/84.61+61.63E 0.7247417562935823
S90.03= 16.64+41.20+27.95E 4.239999999999995
S-37.72= -11.83--27.41+-47.54E 5.759999999999998
S112.38= 17.30+23.30+78.97E 7.189999999999998
S471.40= -7.66*-76.95+-30.70E 87.33699999999999
S-17.67= -49.66/2.63+-1.60E 2.8121292775665374
S3268.23= 43.54*72.95+-67.23E 159.2170000000001
S25.38= 55.42-88.85+62.40E 3.590000000000007
S-134.31= -69.52-27.31+-29.32E 8.159999999999997
S-31.26= -4.47+-56.97+28.35E 1.8299999999999947
S-126.99= -16.59-69.76+-49.76E 9.120000000000019
S-3776.09= 60.69*-60.19+51.56E 174.7189000000003
S-93.66= -98.26-